In [7]:
import pandas as pd
import numpy as np
import joblib
import pickle
from datetime import datetime, timedelta

from google.colab import drive
drive.mount('/content/drive')

class ZoneForecastSystem:
    def __init__(self, kmeans_path, scaler_path, cluster_csv_path, arima_model_path):
        self.kmeans = joblib.load(kmeans_path)
        self.scaler = joblib.load(scaler_path)
        self.cluster_summary = pd.read_csv(cluster_csv_path).groupby('cluster')[['average_latency', 'total_throughput']].mean()
        with open(arima_model_path, 'rb') as f:
            self.fitted_arima = pickle.load(f)['fitted_model']

    def label_zone(self, cluster_id):
        throughput = self.cluster_summary.loc[cluster_id, 'total_throughput']
        latency = self.cluster_summary.loc[cluster_id, 'average_latency']
        avg_throughput = self.cluster_summary['total_throughput'].mean()
        avg_latency = self.cluster_summary['average_latency'].mean()
        if throughput > avg_throughput and latency < avg_latency:
            return 'High Performance'
        elif throughput < avg_throughput and latency > avg_latency:
            return 'Low Performance'
        else:
            return 'Moderate'

    def predict_zone(self, avg_latency, bandwidth, throughput, latitude, longitude):
        input_data = pd.DataFrame([[avg_latency, bandwidth, throughput, latitude, longitude]],
                                  columns=['average_latency', 'total_bandwidth', 'total_throughput', 'latitude', 'longitude'])
        scaled_input = self.scaler.transform(input_data)
        cluster = self.kmeans.predict(scaled_input)[0]
        zone = self.label_zone(cluster)
        return cluster, zone

    def forecast_throughput(self, steps):
        now = datetime.now()
        future_index = pd.date_range(start=now, periods=steps, freq='h')
        exog = pd.DataFrame({
            'hour': future_index.hour,
            'day_of_week': future_index.dayofweek
        }, index=future_index)
        forecast = self.fitted_arima.forecast(steps=steps, exog=exog)
        return forecast, future_index

# Initialize system with your model paths
system = ZoneForecastSystem(
    "/content/drive/MyDrive/Colab Notebooks/Group Assignment/SavedModel/Clustering/kmeans_model.pkl",
    "/content/drive/MyDrive/Colab Notebooks/Group Assignment/SavedModel/Clustering/scaler.pkl",
    "/content/drive/MyDrive/Colab Notebooks/Group Assignment/SavedModel/Clustering/clustered_50k_output.csv",
    "/content/drive/MyDrive/Colab Notebooks/Group Assignment/SavedModel/TimeSeries/arima_model.pkl"
)

# Main Text-Based UI Loop
while True:
    print("\n==== 5G Prediction System ====")
    print("1. Predict Zone (Clustering)")
    print("2. Forecast Throughput (Time Series)")
    print("0. Exit")

    option = input("Choose an option (0, 1, 2): ")

    if option == "1":
        try:
            lat = float(input("Enter Latitude: "))
            lon = float(input("Enter Longitude: "))
            latency = float(input("Enter Average Latency (ms): "))
            bandwidth = float(input("Enter Total Bandwidth: "))
            throughput = float(input("Enter Total Throughput: "))

            cluster, zone = system.predict_zone(latency, bandwidth, throughput, lat, lon)
            print(f"\nPredicted Cluster: {cluster}")
            print(f"Performance Zone: {zone}")
        except ValueError:
            print("Invalid input. Please enter numeric values.")

    elif option == "2":
        try:
            steps = int(input("Enter number of hours to forecast: "))
            forecast, times = system.forecast_throughput(steps)
            print("\nForecasted Throughput:")
            for i in range(len(forecast)):
                print(f"{times[i].strftime('%Y-%m-%d %H:%M')}: {forecast.iloc[i]:.2f}")
        except ValueError:
            print("Invalid input. Please enter a valid number of hours.")

    elif option == "0":
        print("Exiting system. Goodbye!")
        break

    else:
        print("Invalid option. Please choose 0, 1, or 2.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

==== 5G Prediction System ====
1. Predict Zone (Clustering)
2. Forecast Throughput (Time Series)
0. Exit
Choose an option (0, 1, 2): 1
Enter Latitude: -37.7126
Enter Longitude: 144.859
Enter Average Latency (ms): 1000
Enter Total Bandwidth: 121212
Enter Total Throughput: 11111

Predicted Cluster: 0
Performance Zone: High Performance

==== 5G Prediction System ====
1. Predict Zone (Clustering)
2. Forecast Throughput (Time Series)
0. Exit
Choose an option (0, 1, 2): 2
Enter number of hours to forecast: 5

Forecasted Throughput:
2025-05-25 15:05: 35.26
2025-05-25 16:05: 34.90
2025-05-25 17:05: 34.29
2025-05-25 18:05: 33.81
2025-05-25 19:05: 33.40

==== 5G Prediction System ====
1. Predict Zone (Clustering)
2. Forecast Throughput (Time Series)
0. Exit
Choose an option (0, 1, 2): 0
Exiting system. Goodbye!
